In [ ]:
#!/usr/bin/env -S python3 -u
import time
import json

import requests
import os

import yaml



In [ ]:

# Replace 'config.yaml' with the path to your YAML configuration file
config_file_path = 'config.yaml'

with open(config_file_path, 'r') as file:
    config = yaml.safe_load(file)

print(config)


In [ ]:
API_TOKEN = os.environ.get('AFID_TOKEN','')
OUTPUT_FILE = "./coco-export.json"



In [ ]:
target_labels = [
    ("lethrinidae", "lethrinus", "atkinsoni"),
    ("terapontidae", "pelates", "octolineatus"),
]

In [ ]:


query_struct = {
    "op": "OR",
    "args": [
        {
            "op": "AND",
            "args": [
                { "op": "Q", "args": ["family", "iexact", family ] },
                { "op": "Q", "args": ["genus", "iexact", genus ] },
                { "op": "Q", "args": ["species", "iexact", species ] }
            ]
        }
        for family, genus, species in target_labels
    ]
}

def start_export():
    """
    Initiate the export task and return its ID.
    """
    api_url = f"{config['app']['url']}"

    res = requests.post(
        api_url,
        headers={"Authorization": f"Token {API_TOKEN}"},
        json={"filter": query_struct}
    )

    res.raise_for_status()

    return res.json()["task_id"]


def get_export_results(task_id):
    """
    Get the results of a running export task.
    Blocks until the task has completed.
    """
    while True:
        api_url = f"{config['app']['url']}{task_id}"

        res = requests.get(
            api_url,
            headers={"Authorization": f"Token {API_TOKEN}"},
        )

        res.raise_for_status()
        res = res.json()

        if res["status"] == "error":
            raise Exception("Server encountered an error while processing the error.")
        elif res["status"] == "complete":
            return res["results"]

        print(".", end="")
        time.sleep(2)


def main():
    print("Running export...", end="")
    task_id = start_export()
    results = get_export_results(task_id)

    with open(OUTPUT_FILE, "w") as f:
        json.dump(results, f)

    print('Done')

if __name__ == "__main__":
    main()

In [ ]:
GIT_TOKEN = os.environ.get("GITHUB_TOKEN")
images_dir = 'images'
!python3 fetch_coco_images.py ./coco-export.json {GIT_TOKEN} -o {images_dir}

In [ ]:
print(GIT_TOKEN)

In [ ]:
access_key = os.environ.get('AFID_APP_ACCESS_KEY')
secret_key = os.environ.get('AFID_APP_SECRET_KEY')

In [ ]:
print(access_key)

In [ ]:
print(secret_key)